This script allows us to use the **Random Forest** machine learning technique to classify different states of late blight severity. This script was executed using google colab. The dzetsaka: Classification Tool [1] was used as a reference. 


First, it is neccesary to mount our drive folder, where our data imput is saved.

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


It is necesary to import all the libraries and files needed to run the script. 

In [8]:
import sys
sys.path.append('/content/drive/MyDrive/cip)/')

In [7]:
import os
import numpy as np
import pickle
from osgeo import (gdal, ogr)
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import accuracy_index as ai
import proccessing_algo
from sklearn.metrics import confusion_matrix, mean_squared_error,accuracy_score,cohen_kappa_score,classification_report
import pandas as pd
from sklearn import tree

ModuleNotFoundError: ignored

In [ ]:
#Loading Raster and Vector File with the classes
inRaster =['drive/MyDrive/cip/data/20191129_40M_RGBNRE_CUTSP_merge2.TIF',
            'drive/MyDrive/cip/data/20191218_40m_RGBNRE_CUTSP_merge2.tif',
            'drive/MyDrive/cip/data/20191129_50m_RGBNRE_merge2.tif',
            'drive/MyDrive/cip/data/20191218_50m_RGBNRE_merge2.tif']
inVector = ['drive/MyDrive/cip/data/20191129_40M_RGBNRE_CUTSP_seg_TRAIN_train10MP.shp',
            'drive/MyDrive/cip/data/20191218_40m_RGBNRE_CUTSP_TRAIN_train10MP2.shp',
           'drive/MyDrive/cip/data/20191129_50m_RGBNRE_seg_train10G.shp',
           'drive/MyDrive/cip/data/20191218_50m_RGBNRE_seg_train10G.shp']
inField ='CLASS'
inSeed = 0
inClassifier = 'RF'


Enter the number of used bands.

In [ ]:
X = np.array([]).reshape(0, 18)# 12 : Feature Number 
Y = np.array([],dtype=np.uint16).reshape(0, 1)

In [ ]:
#Reading the Raster and Vector File
for i in range(0,len(inRaster)):
#Rasterize the vector file into a raster
    ROI = proccessing_algo.rasterize(inRaster[i], inVector[i], inField)
#Extracting the sample data(X) and its label(Y)
    Xi, Yi = proccessing_algo.get_samples_from_roi(inRaster[i], ROI)
    #print(i)
    #print(Xi[100,0])
    #print(Yi[100,0])
    X = np.append(X, Xi, axis=0)
    Y = np.append(Y, Yi, axis=0)
    

In [ ]:
[n, d] = X.shape
#Number of classes
C = int(Y.max()) 
#Scaling all the sample data. Return the Sample scaled, the vector of the max and min value for each feature
X, M, m = proccessing_algo.scale(X)

In [ ]:
#Splitting the samples into 2/3rd for training and 1/3rd for testing
SPLIT=67
#Trainig set
x = np.array([]).reshape(0, d)
y = np.array([]).reshape(0, 1)
#Testing set
xt = np.array([]).reshape(0, d)
yt = np.array([]).reshape(0, 1)
np.random.seed(inSeed)  
for i in range(C):
    t = np.where((i + 1) == Y)[0]
    nc = t.size
    print(nc)
    ns = int(nc * (SPLIT / float(100)))
    rp = np.random.permutation(nc)
    x = np.concatenate((X[t[rp[0:ns]], :], x))
    xt = np.concatenate((X[t[rp[ns:]], :], xt))
    y = np.concatenate((Y[t[rp[0:ns]]], y))
    yt = np.concatenate((Y[t[rp[ns:]]], yt))

8537
8860
8417
9182
9241
8601
3219
4900
5197
12523
1098
5647
6938
7292
6112
7365
4873
4117
3047
12424
10904


In [ ]:

param_grid = dict(n_estimators=3**np.arange(1, 6), max_features=range(1, x.shape[1], int( np.sqrt(x.shape[1]))))
classifier = RandomForestClassifier()
n_splits = 5
cv = StratifiedKFold(n_splits=n_splits) 
y.shape = (y.size,)
grid = GridSearchCV(classifier,param_grid=param_grid,cv=cv)
grid.fit(x, y)
model = grid.best_estimator_
model.fit(x, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=9,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=243,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
grid.best_params_

{'max_features': 9, 'n_estimators': 243}

In [ ]:
importances = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(x.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

Feature ranking:
1. feature 12 (0.090485)
2. feature 6 (0.076033)
3. feature 7 (0.073208)
4. feature 8 (0.072994)
5. feature 10 (0.067807)
6. feature 5 (0.064396)
7. feature 9 (0.062043)
8. feature 11 (0.059374)
9. feature 13 (0.057436)
10. feature 0 (0.051636)
11. feature 2 (0.049508)
12. feature 3 (0.048667)
13. feature 16 (0.048186)
14. feature 1 (0.044332)
15. feature 17 (0.042923)
16. feature 4 (0.037719)
17. feature 15 (0.026805)
18. feature 14 (0.026449)


In [ ]:
model.score(x, y)

1.0

In [ ]:
yp = model.predict(xt)

In [ ]:
from sklearn.metrics import confusion_matrix, mean_squared_error,accuracy_score,cohen_kappa_score,classification_report
import pandas as pd
cm = pd.DataFrame(confusion_matrix(yt, yp).T, 
                  index = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21'], 
                  columns = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21'])
print("Confusion Matrix")
print(cm)
accuracy_score(yt,yp)


Confusion Matrix
       1     2     3     4     5     6  ...    16   17   18   19    20    21
1   2528    89    62    38    17     9  ...     0    0    0    0     0     0
2    170  2704   158    77    38    11  ...     0    0    0    0     0     0
3     63    75  2405    66    46    17  ...     0    0    0    0     0     0
4     43    45    85  2696    81    61  ...     0    0    0    0     1     0
5      3     5    13    57  2606    56  ...     0    0    0    3     1     1
6      5     4    11    22    48  2477  ...     0    0    0    1     4     0
7      0     1     5     5    12     5  ...     0    0    0    0     0     0
8      2     0     8     4    15    17  ...     1    0    1    0     3     0
9      1     0     7    13    25    33  ...     0    1    0    1     1     0
10     3     0    17    44   102    90  ...    32   16    4   14    47     0
11     0     0     0     0     0     0  ...     0    0    0    0     0     0
12     0     0     1     0    31     9  ...    11    6    9

0.7894275105072021

In [ ]:
cohen_kappa_score(yt,yp)

0.7762821391034052

In [ ]:
target_names =['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21']
print("Classification Report")
print(classification_report(yt,yp,target_names=target_names,digits=4))

Classification Report
              precision    recall  f1-score   support

           1     0.9143    0.8971    0.9056      2818
           2     0.8557    0.9248    0.8889      2924
           3     0.8941    0.8657    0.8797      2778
           4     0.8770    0.8895    0.8832      3031
           5     0.8701    0.8544    0.8622      3050
           6     0.9154    0.8725    0.8934      2839
           7     0.9589    0.8119    0.8793      1063
           8     0.9247    0.8126    0.8650      1617
           9     0.8950    0.8193    0.8555      1716
          10     0.7978    0.8889    0.8409      4133
          11     0.9840    0.6777    0.8026       363
          12     0.8248    0.7575    0.7897      1864
          13     0.6953    0.7393    0.7166      2290
          14     0.6504    0.6863    0.6679      2407
          15     0.6566    0.5706    0.6106      2017
          16     0.5658    0.6063    0.5854      2431
          17     0.7071    0.4636    0.5601      1609
     

In [ ]:
output_log_file = os.path.join('drive/MyDrive/cip/output/'+ 'report_5bands_nred_ndvi_11textures243BM_21.txt')

In [ ]:
#Saving classification report
textf = open(output_log_file, "w")
print('Classification Report', file=textf)
print('\n', file=textf)
print('Confusion Matrix:', file=textf)
print(cm, file=textf)
print('\n', file=textf)
print('Classification Report', file=textf)
print(classification_report(yt,yp,target_names=target_names,digits=4),file=textf)
print('\n', file=textf)
print('Cohens Kappa coefficient:', file=textf)
print(cohen_kappa_score(yt,yp),file=textf)
print('\n', file=textf)
print('Feature ranking::', file=textf)
for f in range(x.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]),file=textf )

textf.close()

In [ ]:
#saveConfusionMatrix
np.savetxt('drive/MyDrive/cip/output/confusion_matrix_5bands_nred_ndvi_11textures243BM_21.txt',cm,delimiter=',',header='Columns=prediction,Lines=reference.',fmt='%1.4d')
#saveModel
output = open('drive/MyDrive/cip/output/model_5bands_nred_ndvi_11textures243BM_21.txt', 'wb')
pickle.dump([model, M, m, 'RF'], output)
output.close()

Image Prediction

In [ ]:
#Load Model
modelfile = open('drive/MyDrive/cip/output/model_5bands_nred_ndvi_5texturespca11.txt', 'rb')
model, M, m, classifier = pickle.load(modelfile)
modelfile.close()

In [ ]:
inRaster =['data/20191129_40M_RGBNRE_CUTSP_merge2.TIF',
            'data/20191218_40m_RGBNRE_CUTSP_merge2.tif',
           'data/20191129_50m_RGBNRE_merge2.tif',
            'data/20191218_50m_RGBNRE_merge2.tif']

In [ ]:
#Applying the Model to a Raster File
raster = gdal.Open(inRaster[0], gdal.GA_ReadOnly)
d = raster.RasterCount
nc = raster.RasterXSize
nl = raster.RasterYSize
GeoTransform = raster.GetGeoTransform()
Projection = raster.GetProjection()

band = raster.GetRasterBand(1)
block_sizes = band.GetBlockSize()
x_block_size = block_sizes[0]
y_block_size = block_sizes[1]
del band
nodatatest = raster.GetRasterBand(1).GetNoDataValue()

In [ ]:
outRaster = 'drive/MyDrive/cip/output/model_5bands_nred_ndvi_5textures_G12.tif'
driver = gdal.GetDriverByName('GTiff')
dst_ds = driver.Create(outRaster, nc, nl, 1, gdal.GDT_Byte)
dst_ds.SetGeoTransform(GeoTransform)
dst_ds.SetProjection(Projection)
out = dst_ds.GetRasterBand(1)
nClass = len(model.classes_)
# Perform the classification
total = nl * y_block_size
for i in range(0, nl, y_block_size):
    if i + y_block_size < nl:  # Check for size consistency in Y
        lines = y_block_size
    else:
         lines = nl - i
    for j in range(0, nc, x_block_size):  # Check for size consistency in X
        if j + x_block_size < nc:
            cols = x_block_size
        else:
            cols = nc - j
        X = np.empty((cols * lines, d))
        for ind in range(d):
            X[:, ind] = raster.GetRasterBand(
                int(ind + 1)).ReadAsArray(j, i, cols, lines).reshape(cols * lines)
        band_temp = raster.GetRasterBand(1)
        nodata_temp = band_temp.GetNoDataValue()
        if nodata_temp is None:
            nodata_temp = -9999
        mask_temp = band_temp.ReadAsArray(j, i, cols, lines).reshape(cols * lines)
        temp_nodata = np.where(mask_temp != nodata_temp)[0]
        t = np.where(X[:, 0] != nodata_temp)[0]
        yp = np.zeros((cols * lines,))
        if t.size > 0:
            yp[t] = model.predict(proccessing_algo.scale(X[t, :], M=M, m=m))
        out.WriteArray(yp.reshape(lines, cols), j, i)
        NODATA=0
        out.SetNoDataValue(NODATA)
        out.FlushCache()
        del X, yp
raster = None
dst_ds = None

References:
[1] Karasiak, N. (2017). Dzetsaka: Classification tool. Retrieved fromhttps://plugins.qgis.org/plugins/dzetsaka